In [2]:
# Import needed packages for analysis
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib

import cfl.util.brain_util as BU
import cfl.util.brain_vis as BV
import cfl.util.fear_mice_functions as fm
from cfl.util.fear_mice_functions import select_from_dict

mri_dir, mri_dims, affine, dir_labels = fm.get_global_values()
behav_csv = 'PTSD_Data_Share\Behavior_data\PTSD_PerLight.csv'

# load the non-linear mask template
nl_mask_path = os.path.join('PTSD_Data_Share/templates\MuseTemplate_nonlinear_mask.nii')
nl_mask = BU.load_brain(nl_mask_path, ori='RPS')
nolin_mask_vec = BU.flatten(nl_mask)

# load response data 
Y = pd.read_pickle('Y.pkl')

In [3]:
# load all the images in RPS orientation 
X, Y_unused = BU.load_data(mri_dir, behav_csv, mri_dims, ori='RPS')

In [4]:
data_id = 'X_'

# Heatmap 

These heatmaps show the average value of each voxel for the group 

## Heatmap with each timepoint broken out (aggregated across both genotypes)

In [10]:
timepoints_dir = fm.timepoint_indices_dir(Y)

HM_list = [] 
for timepoint in timepoints_dir: 
    currentHM = np.zeros(np.prod(mri_dims))
    indices = timepoints_dir[timepoint]
    n = len(timepoints_dir[timepoint])
    for brain in X[indices]:
        currentHM += brain
    np.divide(currentHM, n)
    HM_list.append(currentHM)

    save_name = data_id + 'heatmap_' + timepoint + '.npy'
    np.save(os.path.join('npy_files', save_name), currentHM) 

In [6]:
# load heatmaps 
heatmap_names = [data_id +'heatmap_' + str(timepoint) for timepoint in timepoints_dir.keys()]
HM_list = []
for heatmap in heatmap_names: 
    HM_list.append(np.load(heatmap + '.npy'))

# # display heatmaps 
# BV.plot_interactive_panels(np.vstack(HM_list), mri_dims, nolin_mask_vec, figsize=(17, 3), dir_labels=dir_labels, column_titles=list(timepoints_dir))

FileNotFoundError: [Errno 2] No such file or directory: 'X_heatmap_BL.npy'

The Baseline images are in general slightly darker than the other images. 

My interpretation of this is that, since the baseline images are not Mn(II) enhanced, the high activity peaks are lower relative to the background level of activity, and since all the images have been normalized along the same scale, the 'low activity' or 'normal' activity levels for the Baseline images have a higher numeric value after normalization compared to  the other images. 

"Overall, the greater in magnitude the voxel intensity value of a contrast enhanced image comparing to each animal's baseline non-contrast enhanced image, corresponds to greater Mn(II) accumulation i.e. neural activity in a particular cluster of voxels."

From this statement, I think they may be saving that we should analyze the intensity in a pre-fear/fear/D9 image minus the intensity in the baseline image, in order to find spikes in activity? 

Since the "artifacts" on the edge of the brain are generally 0 (regardless of whether they're from the baseline or not), it seems like the artifacts end up appearing as "neutral" - the activity is about the same between baseline and fear image 

- is this true or are the artifacts negative? 

- how does this show up with individual images (non heatmaps)?

# Heatmaps, WT vs KO 

These heatmaps show the average value of each voxel for the timepoint, by each genotype  

In [12]:
# HM_dir = fm.geno_time_indices_dir(Y)

# for curr in HM_dir: 
#     currentHM = np.zeros(np.prod(mri_dims))
#     indices = HM_dir[curr]
#     n = len(indices)
#     for brain in X[indices]:
#         currentHM += brain
#     np.divide(currentHM, n)
#     HM_dir[curr] = currentHM

#     save_name = data_id + 'heatmap_' + curr + '.npy'
#     np.save(os.path.join('npy_files', save_name), currentHM) 

In [8]:
# load heatmaps for each genotype, timepoint 
all_HM_dir = fm.empty_geno_time_dir(Y, mri_dims)
for key in all_HM_dir: 
    all_HM_dir[key] = np.load(os.path.join('npy_files', data_id + 'heatmap_' + key + '.npy'))

In [6]:
# display WT heatmaps 
wts, wt_names = select_from_dict(all_HM_dir, ['WT'])

BV.plot_interactive_panels(wts, mri_dims, nolin_mask_vec, figsize=(17, 3), dir_labels=dir_labels, column_titles=wt_names)

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

In [30]:
# display KO heatmaps 
kos, ko_names = select_from_dict(all_HM_dir, ['KO'])

# BV.plot_interactive_panels(kos, mri_dims, nolin_mask_vec, figsize=(17, 3), dir_labels=dir_labels, column_titles=ko_names)

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

In [41]:
#display BL and pre-fear heatmaps 
bl_and_pref, bl_and_pref_names = select_from_dict(all_HM_dir, ['BL', 'PreF'])

# BV.plot_interactive_panels(bl_and_pref, mri_dims, nolin_mask_vec, figsize=(17, 3), dir_labels=dir_labels, column_titles=bl_and_pref_names)

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

In [42]:
#display post fear and D9 heatmaps 
fear_and_d9, fear_and_d9_names = select_from_dict(all_HM_dir, ['Fear', 'D9'])

# BV.plot_interactive_panels(fear_and_d9, mri_dims, nolin_mask_vec, figsize=(17, 3), dir_labels=dir_labels, column_titles=fear_and_d9_names)

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

In [9]:
# save the heatmaps as nifti files 
for key in all_HM_dir: 
    name = data_id + 'heatmap_' + key 
    fm.save_as_nifti(all_HM_dir[key], os.path.join('correctly_oriented_niis', name), mri_dims, affine)